<div style="background: #f6ff47; 
            font-size: 20px; 
            padding: 10px; 
            border: 1px solid lightgray; 
            margin: 10px;
            color:#020024;
            text-align:center;
            border-radius: 30px;">
  This is not the Main Notebook. Please check out <img src="https://media.giphy.com/media/WUlplcMpOCEmTGBtBW/giphy.gif" width="30">
    <br>
    <a  href="https://www.kaggle.com/soumya9977/attention-based-residual-unet-eda-on-cell-imgs">Attention based Residual Unet + EDA on cell imgs🧬</a> 
</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm_notebook as tqdm
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
train_img = "../input/sartorius-cell-instance-segmentation/train"
train_csv = "../input/sartorius-cell-instance-segmentation/train.csv"

df = pd.read_csv(train_csv)
df.head()

In [ ]:
np.unique(df["id"])

In [ ]:
df["annotation"][0]

In [ ]:
df["cell_type"].value_counts()

In [ ]:
labels = df[df["id"] == "0030fd0e6378"]["annotation"].tolist()
labels

In [ ]:
df[df["id"] == "0030fd0e6378"]["cell_type"].value_counts()

In [ ]:
len(np.unique(df["id"])) # this shows that there are only 606 unique images

In [ ]:
len(df["id"])

In [ ]:
s = labels[0].split()
print(len(s))
s

In [ ]:
starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
lengths

In [ ]:
shape = (520, 704,3)
img = np.zeros((shape[0]* shape[1],shape[2]), dtype=np.float32)

In [ ]:
starts -= 1
ends = starts + lengths
for lo, hi in zip(starts, ends):
        img[lo : hi] = 1.0

In [ ]:
print(starts)
print(ends)

In [ ]:
plt.imshow(img.reshape(shape))

In [ ]:
np.unique(img)

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)
    



def build_masks(image_id,input_shape, colors=True):
    height, width = input_shape
    labels = df[df["id"] == image_id]["annotation"].tolist()
    if colors:
        mask = np.zeros((height, width, 3))
        for label in labels:
            mask += rle_decode(label, shape=(height,width , 3), color=np.random.rand(3))
    else:
        mask = np.zeros((height, width, 1))
        for label in labels:
            mask += rle_decode(label, shape=(height, width, 1))
    mask = mask.clip(0, 1)
    return mask
    

In [ ]:
sample_filename = '115fad550598'
sample_image_df = df[df['id'] == sample_filename]
sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
sample_img = plt.imread(sample_path)
sample_rles = sample_image_df['annotation'].values

COLOR_MAP = 'seismic'
sample_masks1=build_masks(sample_filename,input_shape=(520, 704), colors=False)
sample_masks2=build_masks(sample_filename,input_shape=(520, 704), colors=True)

# sample_filename1 = '172334437c88'
# sample_image_df1 = df[df['id'] == sample_filename1]
# sample_path1 = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df1['id'].iloc[0]}.png"
# sample_img1 = plt.imread(sample_path1)
# sample_rles1 = sample_image_df1['annotation'].values

# COLOR_MAP = 'seismic'
# sample_masks3=build_masks(sample_filename1,input_shape=(520, 704), colors=False)
# sample_masks4=build_masks(sample_filename1,input_shape=(520, 704), colors=True)






fig, axs = plt.subplots(1,3, figsize=(18, 18))
axs[0].imshow(sample_img,cmap = 'seismic')
axs[0].set_title('original image', fontsize=16)


axs[1].imshow(sample_masks1,cmap='gray')
axs[1].set_title('mask image', fontsize=16)


axs[2].imshow(sample_masks2)
axs[2].set_title('mask with color', fontsize=16)

# axs[3].imshow(sample_img1)
# # axs[2].set_title('mask with color', fontsize=16)

# axs[4].imshow(sample_masks3)

# axs[5].imshow(sample_masks4)

In [ ]:
sample_filename = '115fad550598'
sample_image_df = df[df['id'] == sample_filename]
sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
sample_img = plt.imread(sample_path)
# sample_rles = sample_image_df['annotation'].values

COLOR_MAP = 'seismic'
sample_masks1=build_masks(sample_filename,input_shape=(520, 704), colors=False)
sample_masks2=build_masks(sample_filename,input_shape=(520, 704), colors=True)


sample_filename1 = '172334437c88'
sample_image_df1 = df[df['id'] == sample_filename1]
sample_path1 = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df1['id'].iloc[0]}.png"
sample_img1 = plt.imread(sample_path1)
# sample_rles1 = sample_image_df1['annotation'].values

# COLOR_MAP = 'seismic'
sample_masks3=build_masks(sample_filename1,input_shape=(520, 704), colors=False)
sample_masks4=build_masks(sample_filename1,input_shape=(520, 704), colors=True)


plt.figure(figsize=(14,14))
plt.subplot(2, 3, 1)
plt.imshow(sample_img,cmap = 'seismic')

plt.subplot(2, 3, 2)
plt.imshow(sample_masks1,cmap='gray')

plt.subplot(2, 3, 3)
plt.imshow(sample_masks2)



plt.subplot(2, 3, 4)
plt.imshow(sample_img1,cmap = 'seismic')

plt.subplot(2, 3, 5)
plt.imshow(sample_masks3,cmap='gray')

plt.subplot(2, 3, 6)
plt.imshow(sample_masks4)

# fig, axs = plt.subplots(1,3, figsize=(18, 18))
# axs[0].imshow(sample_img,cmap = 'seismic')
# axs[0].set_title('original image', fontsize=16)


# axs[1].imshow(sample_masks1,cmap='gray')
# axs[1].set_title('mask image', fontsize=16)


# axs[2].imshow(sample_masks2)
# axs[2].set_title('mask with color', fontsize=16)

# axs[3].imshow(sample_img1)
# # axs[2].set_title('mask with color', fontsize=16)

# axs[4].imshow(sample_masks3)

# axs[5].imshow(sample_masks4)

In [ ]:
# img = np.expand_dims(sample_img, axis=-1)
# y = np.squeeze(y, axis=-1)
# img.shape

In [ ]:
from sklearn.model_selection import train_test_split
train_idx, val_idx = train_test_split(
    df.index, random_state=2019, test_size=0.2 # mask_count_df
)

print(train_idx)

In [ ]:
sample_img.shape

In [ ]:
img_new = np.zeros((520, 704,3),dtype=np.int8)
img_new[:,:,0] = sample_masks1[:,:,0]
img_new[:,:,1] = sample_masks1[:,:,0]
img_new[:,:,2] = sample_masks1[:,:,0]
img_new = img_new * 255
# np.unique(img_new)
# plt.imshow(img_new)
np.where(img_new == [255, 255, 255])

In [ ]:
sample_img

In [ ]:
temp = np.zeros((520, 704,3),dtype=np.float32)
# temp = np.zeros_like(thresh1,dtype=np.float32)
temp[:,:,0] = sample_img[:,:]
temp[:,:,1] = sample_img[:,:]
temp[:,:,2] = sample_img[:,:]
# cv2.imwrite('temp_img.jpg', temp)
# temp = temp *255
# norm_image = cv2.normalize(temp, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

plt.imshow(temp,cmap='gray')

In [ ]:
# !pip install opencv-python==4.4.0.40

In [ ]:
color=[0, 242, 214]
# read the mask
mask = img_new
# read the original img
img = temp
# change the color coden to light red
ret, thresh1 = cv2.threshold(mask, 120, 255, cv2.THRESH_BINARY)
check = np.array([255, 255, 255])
thresh1[np.where((thresh1==check).all(axis=0))] = color
# overlay the mask
thresh1 =thresh1.astype(np.float32)
combine = cv2.addWeighted(img,0.7,thresh1,0.7,0.3)
# combine = cv2.add(img,thresh1)
plt.imshow(combine)
plt.show()

## Data Preprocessing

In [ ]:
import numpy as np
unq_id = np.unique(df["id"])

# if os.path.exists("./images")
# os.mkdir("./images")
os.mkdir("./masks")
# os.mkdir("./images/img")
os.mkdir("./masks/img")
print(len(unq_id))
cnt = 0
for ide in tqdm(unq_id):
#     print(i)
#     for ide in ids:
    sample_image_df = df[df['id'] == ide]
    sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
    sample_img = plt.imread(sample_path)
    sample_masks1=build_masks(ide,input_shape=(520, 704), colors=False)
    
#     sample_image_df1 = df[df['id'] == i]
#     sample_path1 = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df1['id'].iloc[0]}.png"
#     sample_img1 = plt.imread(sample_path1)
#     sample_masks3=build_masks(sample_filename1,input_shape=(520, 704), colors=False)
    
#     sample_masks1=build_masks(ide,input_shape=(520, 704), colors=False)
    
#     sample_image_df = df[df['id'] == ide]
#     sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
#     sample_img = cv2.imread(sample_path,cv2.IMREAD_COLOR)
    
#     img_file = f"./images/img/{sample_image_df1['id'].iloc[0]}.png"
    mask_file = f"./masks/img/{sample_image_df['id'].iloc[0]}.png"
    print(mask_file)
#     cv2.imwrite(img_file,sample_img1)
    cv2.imwrite(mask_file,sample_masks1)

    cnt+=1
print(f"preprocessed total data {cnt}")

In [ ]:
# !cd ..

In [ ]:
# !pwd

In [ ]:
# !dir masks/img

In [ ]:
im = plt.imread(os.path.join("./masks/img", os.listdir("./masks/img")[17]))
plt.imshow(im)

In [ ]:
imr = plt.imread("./masks/img/fac5fa0dd448.png")
plt.imshow(imr)